all datasets that will be used require adding additional information to serve the purpose of the semantic data integration excercise course purpose
while heterogenity already exists between all three selected datasets. the use case is Electronic Health Records (EHR), which would require adding additional information linked to patients. 



##requires decoding according to the provided codebook to get back the raw data 
https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset


requires transforming from .data file to .csv.
https://archive.ics.uci.edu/dataset/45/heart+disease

requires adding generated information 
https://www.kaggle.com/datasets/aasheesh200/framingham-heart-study-dataset/code

final datasets:
- farmingham_dirty.csv / simulating a cargioligist check-up
- heart_dirty.csv / simulating a cargioligist check-up
- diabetes_dirty.csv / simulating a endocrinologist check-up

In [ ]:
#### Heart disease data, from .data to .csv

import pandas as pd
import re

files = [
    "cleveland.data",
    "hungarian.data",
    "switzerland.data",
    "long-beach-va.data"
]
output_file = "heart_disease_full_raw.csv"

uci_columns = [
    "id", "ccf", "age", "sex", "painloc", "painexer", "relrest", "pncaden",
    "cp", "trestbps", "htn", "chol", "smoke", "cigs", "years", "fbs", "dm",
    "famhist", "restecg", "ekgmo", "ekgday", "ekgyr", "dig", "prop", "nitr",
    "pro", "diuretic", "proto", "thaldur", "thaltime", "met", "thalach",
    "thalrest", "tpeakbps", "tpeakbpd", "dummy", "trestbpd", "exang", "xhypo",
    "oldpeak", "slope", "rldv5", "rldv5e", "ca", "restckm", "exerckm",
    "restef", "restwm", "exeref", "exerwm", "thal", "thalach", "tpeakrate",
    "dummy2", "trestef", "trestwm", "exeref", "exerwm", "thalsev", "thalpul",
    "earlobe", "cmo", "cday", "cyr", "num", "lmt", "ladprox", "laddist",
    "diag", "cxmain", "ramus", "om1", "om2", "rcaprox", "rcadist", "name"
]

expected_columns = len(uci_columns)  

records = []
current_record = ""

for file in files:
    with open(file, "r", encoding="latin1") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            current_record += " " + line
            if line.endswith("name"):
                fields = re.split(r"\s+", current_record.strip())
                if len(fields) < expected_columns:
                    fields += [""] * (expected_columns - len(fields))
                elif len(fields) > expected_columns:
                    fields = fields[:expected_columns]
                records.append(fields)
                current_record = ""

df = pd.DataFrame(records, columns=uci_columns)


df.to_csv(output_file, index=False)



## change IDs (redundant. two IDs per patient)
##


In [ ]:
## mapping diabetes binary health indicators to the original values based on the cookbook: https://www.cdc.gov/brfss/annual_data/2015/pdf/codebook15_llcp.pdf
## data transformations
import pandas as pd
df = pd.read_csv("diabetes_binary_health_indicators_BRFSS2015.csv")


codebook_mappings = {
    "Diabetes_binary": {0.0: "No", 1.0: "Yes"},
    "HighBP": {0.0: "No", 1.0: "Yes"},
    "HighChol": {0.0: "No", 1.0: "Yes"},
    "CholCheck": {0.0: "No", 1.0: "Yes"},
    "Smoker": {0.0: "No", 1.0: "Yes"},
    "Stroke": {0.0: "No", 1.0: "Yes"},
    "HeartDiseaseorAttack": {0.0: "No", 1.0: "Yes"},
    "PhysActivity": {0.0: "No", 1.0: "Yes"},
    "Fruits": {0.0: "No", 1.0: "Yes"},
    "Veggies": {0.0: "No", 1.0: "Yes"},
    "HvyAlcoholConsump": {0.0: "No", 1.0: "Yes"},
    "AnyHealthcare": {0.0: "No", 1.0: "Yes"},
    "NoDocbcCost": {0.0: "No", 1.0: "Yes"},
    "GenHlth": {
        1.0: "Excellent",
        2.0: "Very good",
        3.0: "Good",
        4.0: "Fair",
        5.0: "Poor"
    },
    "DiffWalk": {0.0: "No", 1.0: "Yes"},
    "Sex": {0.0: "Female", 1.0: "Male"},
    "Education": {
        1.0: "Never attended/Kindergarten",
        2.0: "Grades 1-8",
        3.0: "Grades 9-11",
        4.0: "High school graduate",
        5.0: "Some college",
        6.0: "College graduate"
    },
    "Income": {
        1.0: "<$10,000",
        2.0: "$10,000-$15,000",
        3.0: "$15,000-$20,000",
        4.0: "$20,000-$25,000",
        5.0: "$25,000-$35,000",
        6.0: "$35,000-$50,000",
        7.0: "$50,000-$75,000",
        8.0: "$75,000+"
    },
    "Age": {
        1.0: "18-24",
        2.0: "25-29",
        3.0: "30-34",
        4.0: "35-39",
        5.0: "40-44",
        6.0: "45-49",
        7.0: "50-54",
        8.0: "55-59",
        9.0: "60-64",
        10.0: "65-69",
        11.0: "70-74",
        12.0: "75-79",
        13.0: "80+"
    }
}

for col, mapping in codebook_mappings.items():
    if col in df.columns:
        df[col] = df[col].map(mapping)

output_path = "decoded_brfss2015.csv"
df.to_csv(output_path, index=False)




Generating Patient id and patient name, while ensuring the existence of case of heterognity in values. ( patient having the same name, but is written differently in different datasets)

In [ ]:
import pandas as pd
import numpy as np
import random
from faker import Faker

diabetes = pd.read_csv("decoded_brfss2015.csv")
heart = pd.read_csv("heart_disease_full_raw.csv")
framingham = pd.read_csv("framingham.csv")  

faker = Faker()
random.seed(26)
Faker.seed(26)

n_patients = 1000
patients = pd.DataFrame({
    "patient_id": range(1, n_patients + 1),
    "canonical_name": [faker.name() for _ in range(n_patients)]
})

##LLM Generated
def generate_name_variant(name):
    parts = name.split()
    first = parts[0]
    last = parts[-1]
    middle = parts[1] if len(parts) > 2 else ""

    variants = [
        f"{first} {last}",
        f"{first[0]}. {last}",
        f"{first} {last[0]}.",
        f"{first} {middle[0]}. {last}" if middle else f"{first} {faker.random_letter().upper()}. {last}",
        f"{faker.first_name()} {last}",
        f"{first} {faker.last_name()}",
        f"{first} {last}".upper(),
        f"{first.lower()} {last.lower()}"
    ]
    return random.choice(variants)
####
patients["name_diabetes"] = patients["canonical_name"].apply(generate_name_variant)
patients["name_heart"] = patients["canonical_name"].apply(generate_name_variant)
patients["name_framingham"] = patients["canonical_name"].apply(generate_name_variant)

overlap = 300
shared = patients.sample(n=overlap, random_state=1)

diabetes_patients = pd.concat([shared, patients.iloc[overlap:overlap+350]]).reset_index(drop=True)
heart_patients = pd.concat([shared, patients.iloc[overlap+100:overlap+450]]).reset_index(drop=True)
framingham_patients = pd.concat([shared, patients.iloc[overlap+200:overlap+500]]).reset_index(drop=True)

def inject_identity(df, patient_subset, name_col):
    df = df.copy()
    df = df.head(len(patient_subset)).reset_index(drop=True)
    df["patient_id"] = patient_subset["patient_id"].values
    df["name"] = patient_subset[name_col].values
    return df

diabetes_dirty = inject_identity(diabetes, diabetes_patients, "name_diabetes")
heart_dirty = inject_identity(heart, heart_patients, "name_heart")
framingham_dirty = inject_identity(framingham, framingham_patients, "name_framingham")

diabetes_dirty.to_csv("diabetes_ready.csv", index=False)
heart_dirty.to_csv("heart_ready.csv", index=False)
framingham_dirty.to_csv("framingham_ready.csv", index=False)



Creation of data dictionary

In [2]:
import pandas as pd

diabetes_df = pd.read_csv('diabetes_ready.csv')
framingham_df = pd.read_csv('framingham_ready.csv')
heart_df = pd.read_csv('heart_ready.csv')

def classify_data_type(dtype, series):
    if pd.api.types.is_numeric_dtype(series):
        unique_vals = series.nunique()
        if unique_vals < 15:
            return "categorical"
        else:
            return "numerical"
    elif pd.api.types.is_string_dtype(series):
        return "string"
    else:
        return "categorical"

def create_classified_data_dictionary(df, dataset_name):
    data = []
    for col in df.columns:
        example_value = df[col].dropna().iloc[0] if not df[col].dropna().empty else ""
        classification = classify_data_type(df[col].dtype, df[col])
        data.append((dataset_name, col, classification, example_value))
    return pd.DataFrame(data, columns=["dataset", "column_name", "data_type", "example_value"])

diabetes_dict = create_classified_data_dictionary(diabetes_df, "diabetes")
framingham_dict = create_classified_data_dictionary(framingham_df, "framingham")
heart_dict = create_classified_data_dictionary(heart_df, "heart")

combined_dict = pd.concat([diabetes_dict, framingham_dict, heart_dict], ignore_index=True)

combined_dict.to_csv("data_dictionary.csv", index=False)
